# Multi-Agentic Financial Analyst
This notebook will be the test build of the multi-agentic AI system. This is a notebook to experiment with the solution and to 

In [7]:
#%package install boto3=1.34.51 langchain 

In [1]:
import bql
import boto3
import botocore
import random
import json
import importlib
from utils.s3_helper import S3Helper

from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence
from langchain_core.rate_limiters import InMemoryRateLimiter

from langchain_aws import ChatBedrock
from pydantic import BaseModel, Field
from IPython.display import Markdown, display
import company_data
from datetime import datetime
from dateutil.relativedelta import relativedelta

import prompts
import pandas as pd

import concurrent.futures

In [2]:
bq = bql.Service()

In [3]:
importlib.reload(prompts)

<module 'prompts' from '/project/prompts.py'>

In [4]:
# Bedrock client initialise
config = botocore.config.Config(read_timeout=1000)
boto3_bedrock = boto3.client('bedrock-runtime',config=config)

with open('Data/prompts.json', 'rb') as f:
    prompt_set = json.load(f)

### Configure the model

In [5]:
model_claude_id = 'us.anthropic.claude-3-7-sonnet-20250219-v1:0'
model_claude_small = "us.anthropic.claude-3-5-haiku-20241022-v1:0"
model_llama_id = 'us.meta.llama3-1-70b-instruct-v1:0'
model_llama_small_id = "us.meta.llama3-1-8b-instruct-v1:0"

# set up the LLM in Bedrock
rate_limiter = InMemoryRateLimiter(
    requests_per_second=50,
    check_every_n_seconds=1,
    max_bucket_size=10,
)

llm = ChatBedrock(
    client = boto3_bedrock,
    model_id = model_claude_id,
    temperature = 0.01,
    max_tokens=4000,
    rate_limiter = rate_limiter
)

llm_small = ChatBedrock(
    client = boto3_bedrock,
    model_id = model_llama_small_id,
    temperature = 0.01,
    max_tokens = 4000,
    rate_limiter = rate_limiter
)

llm_debate = ChatBedrock(
    client = boto3_bedrock,
    model_id = model_claude_small,
    temperature = 0.01,
    max_tokens = 4000,
    rate_limiter = rate_limiter
)

In [6]:
# test prompt with a single security
security = prompt_set[0]
sec_name = security['security']
sec_date = security['date']
sec_fs = security['prompt'][1]['content']
sec_prices = sec_fs[sec_fs.find('Historical Price:') + 17:]

In [7]:
system_prompt = prompts.SYSTEM_PROMPTS['COT_CLAUDE_TEST']['prompt']
prompt_template = PromptTemplate.from_template(system_prompt)
# Create the prompt to feed into the model
prompt_in = prompt_template.format(financials=sec_fs)

In [8]:
output_financial_statements = llm.invoke(prompt_in)


In [9]:
output_financial_statements.content
#display(Markdown(output.content))

"# Financial Analysis Report\n\n## 1. Analysis of Current Profitability, Liquidity, Solvency, and Efficiency Ratios\n\n### Profitability Ratios\n- **Gross Profit Margin**: 21.24% (t) vs 21.61% (t-1)\n- **Operating Profit Margin**: 3.06% (t) vs 3.25% (t-1)\n- **Net Profit Margin**: 2.52% (t) vs 2.69% (t-1)\n- **Return on Assets (ROA)**: 3.88% (t) vs 4.08% (t-1)\n- **Return on Equity (ROE)**: 14.35% (t) vs 15.23% (t-1)\n\n### Liquidity Ratios\n- **Current Ratio**: 0.66 (t) vs 0.66 (t-1)\n- **Quick Ratio**: 0.32 (t) vs 0.31 (t-1)\n- **Cash Ratio**: 0.03 (t) vs 0.03 (t-1)\n\n### Solvency Ratios\n- **Debt-to-Equity Ratio**: 2.70 (t) vs 2.73 (t-1)\n- **Debt-to-Assets Ratio**: 0.73 (t) vs 0.73 (t-1)\n- **Interest Coverage Ratio**: 6.15 (t) vs 6.31 (t-1)\n\n### Efficiency Ratios\n- **Asset Turnover Ratio**: 1.54 (t) vs 1.51 (t-1)\n- **Inventory Turnover**: 11.32 (t) vs 10.22 (t-1)\n- **Receivables Turnover**: 18.32 (t) vs 18.48 (t-1)\n\n## 2. Time-Series Analysis Across Ratios\n\n### Profitabi

# Analyse the News datasets

In [168]:
s3_helper = S3Helper('tmp/fs')
s3_helper.get_file(filename='dow_headlines.parquet', local_filename='/tmp/dow_headlines.parquet')
news_headlines = pd.read_parquet('/tmp/dow_headlines.parquet')

In [13]:
news_headlines

,SUID,Headline,TimeOfArrival,Assigned_ID_BB_GLOBAL
0,PKP0NPDWRGG0,*jpmorgan rehires ling zhang from bgi genomics,2019-01-02 07:26:13.641,BBG000DMBXR2
1,PKP0NQDWRGG0,*jpmorgan names zhang china healthcare investm...,2019-01-02 07:26:14.600,BBG000DMBXR2
2,PKP0PY6KLVR4,apple remains core tech holding in ‘risk-off’ ...,2019-01-02 07:27:34.229,BBG000B9XRY4
3,PKP4IB6K50XT,refinery outages: exxon beaumont; pes philadel...,2019-01-02 08:49:23.087,BBG000GZQ728
4,PKP72M6K50XU,taiwan walks tightrope between china and not c...,2019-01-02 09:44:46.220,BBG000B9XRY4
...,...,...,...,...
104203,SV3C9WBKHDS1,*disney moves date of new 'star wars' movie up...,2025-04-21 23:08:20.275,BBG000BH4R78
104204,SV3C9WBKHDS2,*disney comments on film release schedule in e...,2025-04-21 23:08:20.287,BBG000BH4R78
104205,SV3FAKT0G1KW,"amazon must negotiate with teamsters, labor bo...",2025-04-22 00:13:32.732,BBG000BVPV84
104206,SV3MJUGENSW0,unitedhealth cut to hold at hsbc; pt $490,2025-04-22 02:50:18.840,BBG000CH5208


In [14]:
# Convert sec_name to figi
def convert_to_figi(sec_name: str) -> str:
    """Function to convert Bloomberg tickers in a dataframe to FIGIs for ESL"""
    univ      = sec_name
    field     = {'figi': bq.data.composite_id_bb_global(), 
                 'name': bq.data.name(), 
                 'sector': bq.data.bics_level_1_sector_name()}
    req = bql.Request(univ, field)
    data = bq.execute(req)
    return (data[0].df().loc[sec_name]['figi'], data[1].df().loc[sec_name]['name'], data[2].df().loc[sec_name]['sector'])
    

In [15]:
figi_name, name, sector = convert_to_figi(sec_name)
figi_name

'BBG000BWLMJ4'

In [16]:
sector

'Consumer Staples'

In [17]:

def filter_by_company_date(dataset, security, max_date = None):
    
    if max_date != None:
        #claculate the minimum date to get a 3 month window
        min_date = datetime.strptime(max_date,"%Y-%m-%d") + relativedelta(months=-3)
        min_date = min_date.strftime("%Y-%m-%d")
        filtered_dataset = dataset[(dataset['TimeOfArrival'] < max_date) &  (dataset['TimeOfArrival'] >= min_date) & (dataset['Assigned_ID_BB_GLOBAL'] == security)]
    else:
        filtered_dataset = dataset[dataset['Assigned_ID_BB_GLOBAL'] == security]
    
    filtered_list = filtered_dataset['Headline'].to_list()

    if len(filtered_list) >= 50:
        return random.sample(filtered_list, 50)
    else:
        return filtered_list
    

In [18]:
sec_date

'2020-04-02'

In [19]:
headline_filter_test = filter_by_company_date(news_headlines, figi_name, sec_date)

In [20]:
headline_filter_test

['preview walgreens 1q: awaiting direction on long-term strategy',
 '*walgreens boots 1q adj eps $1.37, est. $1.41',
 '*walgreens boots 1q net sales $34.3b, est. $34.58b',
 'walgreens boots first quarter adjusted eps misses estimates',
 '*walgreens boots alliance maintains fy adjusted eps guidance',
 '*wba 1q us retail pharmacy comp sales +1.6%, est. +5.3%',
 'walgreens sinks 6% pre-mkt; likely to break uptrend',
 'walgreens stock slump deepens as drugstore giant’s profits slide',
 '*walgreens ceo pessina says company had slow start to 2020',
 'walgreens stock slump deepens as drugstore’s profits slide (1)',
 'walgreens off to slow start in 2020; investors skeptical on lbo',
 'walgreens boots pt cut to $56 from $59 at mizuho securities usa',
 'walgreens boots maintains quarterly dividend at 45.75 cents/shr',
 '*walgreens names richard ashworth president of walgreens',
 'walgreens boots goes ex-dividend, trades without payout',
 'walgreens boots implied volatility surges as shares fall'

In [21]:
system_prompt = """You are an assistant to a financial analyst analyzing {security} You must remove any reference to {security} and their products from the following list of headlines and replace them with the term 'blah'. Replace the names of any people such as ceo in the article with the term 'whah' do not refer to {security} at all in your answer:{headlines}"""
prompt_template = PromptTemplate.from_template(system_prompt)
# Create the prompt to feed into the model
prompt_in = prompt_template.format(headlines=headline_filter_test, security=name)

In [22]:
prompt_in

"You are an assistant to a financial analyst analyzing Walgreens Boots Alliance Inc You must remove any reference to Walgreens Boots Alliance Inc and their products from the following list of headlines and replace them with the term 'blah'. Replace the names of any people such as ceo in the article with the term 'whah' do not refer to Walgreens Boots Alliance Inc at all in your answer:['preview walgreens 1q: awaiting direction on long-term strategy', '*walgreens boots 1q adj eps $1.37, est. $1.41', '*walgreens boots 1q net sales $34.3b, est. $34.58b', 'walgreens boots first quarter adjusted eps misses estimates', '*walgreens boots alliance maintains fy adjusted eps guidance', '*wba 1q us retail pharmacy comp sales +1.6%, est. +5.3%', 'walgreens sinks 6% pre-mkt; likely to break uptrend', 'walgreens stock slump deepens as drugstore giant’s profits slide', '*walgreens ceo pessina says company had slow start to 2020', 'walgreens stock slump deepens as drugstore’s profits slide (1)', 'walg

In [23]:
output_cleaned = llm_small.invoke(prompt_in)

In [24]:
output_cleaned

AIMessage(content="Here is the list of headlines with all references to Walgreens Boots Alliance Inc and their products removed:\n\n* 'Preview blah 1Q: awaiting direction on long-term strategy'\n* 'blah 1Q adj eps $1.37, est. $1.41'\n* 'blah 1Q net sales $34.3b, est. $34.58b'\n* 'blah first quarter adjusted eps misses estimates'\n* 'blah 1Q US retail pharmacy comp sales +1.6%, est. +5.3%'\n* 'blah sinks 6% pre-mkt; likely to break uptrend'\n* 'blah stock slump deepens as drugstore giant’s profits slide'\n* 'blah ceo whah says company had slow start to 2020'\n* 'blah stock slump deepens as drugstore’s profits slide (1)'\n* 'blah off to slow start in 2020; investors skeptical on lbo'\n* 'blah pt cut to $56 from $59 at mizuho securities usa'\n* 'blah maintains quarterly dividend at 45.75 cents/shr'\n* 'blah names richard ashworth president of blah'\n* 'blah goes ex-dividend, trades without payout'\n* 'blah implied volatility surges as shares fall'\n* 'blah directed consultants to remove d

In [25]:
output_ceo = llm.invoke("Guess the name of the company blah from the following headline: blah ceo whah says company had slow start to 2020")

In [26]:
output_ceo

AIMessage(content='Based on the headline "blah ceo whah says company had slow start to 2020," I can see that the company name is meant to be "blah" since it appears in the position where a company name would typically be placed in a headline, followed by "ceo" and the CEO\'s name "whah."', additional_kwargs={'usage': {'prompt_tokens': 36, 'completion_tokens': 72, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 108}, 'stop_reason': 'end_turn', 'thinking': {}, 'model_id': 'us.anthropic.claude-3-7-sonnet-20250219-v1:0', 'model_name': 'us.anthropic.claude-3-7-sonnet-20250219-v1:0'}, response_metadata={'usage': {'prompt_tokens': 36, 'completion_tokens': 72, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 108}, 'stop_reason': 'end_turn', 'thinking': {}, 'model_id': 'us.anthropic.claude-3-7-sonnet-20250219-v1:0', 'model_name': 'us.anthropic.claude-3-7-sonnet-20250219-v1:0'}, id='run-d41d92ca-4931-47a6-89fb-c44f6dd3a24a-0', usage_metada

In [27]:
cleaned_headlines = output_cleaned.content.split('\n')

In [28]:
cleaned_headlines[1:]

['',
 "* 'Preview blah 1Q: awaiting direction on long-term strategy'",
 "* 'blah 1Q adj eps $1.37, est. $1.41'",
 "* 'blah 1Q net sales $34.3b, est. $34.58b'",
 "* 'blah first quarter adjusted eps misses estimates'",
 "* 'blah 1Q US retail pharmacy comp sales +1.6%, est. +5.3%'",
 "* 'blah sinks 6% pre-mkt; likely to break uptrend'",
 "* 'blah stock slump deepens as drugstore giant’s profits slide'",
 "* 'blah ceo whah says company had slow start to 2020'",
 "* 'blah stock slump deepens as drugstore’s profits slide (1)'",
 "* 'blah off to slow start in 2020; investors skeptical on lbo'",
 "* 'blah pt cut to $56 from $59 at mizuho securities usa'",
 "* 'blah maintains quarterly dividend at 45.75 cents/shr'",
 "* 'blah names richard ashworth president of blah'",
 "* 'blah goes ex-dividend, trades without payout'",
 "* 'blah implied volatility surges as shares fall'",
 "* 'blah directed consultants to remove damaging findings: nyt'",
 "* 'blah had consultants remove findings about mistake

In [29]:
system_prompt = """You are a financial analyst and are reviewing news for company called blah over the last three months. Blah is in the {sector} sector. Start by listing the revenue drivers for the sector. Then look through the below headlines and determine if blah will see an increase or decrease in their earnings over the next quarter. Think through your response. {headlines}"""
prompt_template = PromptTemplate.from_template(system_prompt)
# Create the prompt to feed into the model
prompt_in = prompt_template.format(headlines=cleaned_headlines[1:], sector=sector)

In [30]:
prompt_in

'You are a financial analyst and are reviewing news for company called blah over the last three months. Blah is in the Consumer Staples sector. Start by listing the revenue drivers for the sector. Then look through the below headlines and determine if blah will see an increase or decrease in their earnings over the next quarter. Think through your response. [\'\', "* \'Preview blah 1Q: awaiting direction on long-term strategy\'", "* \'blah 1Q adj eps $1.37, est. $1.41\'", "* \'blah 1Q net sales $34.3b, est. $34.58b\'", "* \'blah first quarter adjusted eps misses estimates\'", "* \'blah 1Q US retail pharmacy comp sales +1.6%, est. +5.3%\'", "* \'blah sinks 6% pre-mkt; likely to break uptrend\'", "* \'blah stock slump deepens as drugstore giant’s profits slide\'", "* \'blah ceo whah says company had slow start to 2020\'", "* \'blah stock slump deepens as drugstore’s profits slide (1)\'", "* \'blah off to slow start in 2020; investors skeptical on lbo\'", "* \'blah pt cut to $56 from $59 

In [31]:
output_news = llm.invoke(prompt_in)

In [32]:
display(Markdown(output_news.content))

# Financial Analysis of Blah (Consumer Staples Sector)

## Revenue Drivers for the Consumer Staples Sector

1. **Consumer Spending Patterns**: Essential household products and food items that maintain relatively stable demand
2. **Pricing Power**: Ability to pass on cost increases to consumers
3. **Product Innovation**: New product launches and brand extensions
4. **Demographics**: Aging population trends affecting healthcare and personal care products
5. **Private Label Competition**: Impact of store brands on branded products
6. **Distribution Channels**: Expansion of e-commerce and omnichannel strategies
7. **International Expansion**: Growth in emerging markets
8. **Health and Wellness Trends**: Increasing consumer focus on health-related products
9. **Promotional Activity**: Impact of discounting and promotional strategies
10. **Supply Chain Efficiency**: Cost management and operational improvements

## Analysis of Blah's Earnings Outlook

Based on the headlines provided, I anticipate Blah will likely see a **decrease** in earnings over the next quarter. Here's my reasoning:

### Negative Indicators:
1. **Recent Earnings Miss**: 
   - "blah 1Q adj eps $1.37, est. $1.41"
   - "blah 1Q net sales $34.3b, est. $34.58b"
   - "blah first quarter adjusted EPS misses estimates"

2. **Underperforming Retail Metrics**:
   - "blah 1Q US retail pharmacy comp sales +1.6%, est. +5.3%" (significantly below expectations)

3. **Stock Performance Issues**:
   - "blah sinks 6% pre-mkt; likely to break uptrend"
   - "blah stock slump deepens as drugstore giant's profits slide"
   - Multiple mentions of implied volatility increases

4. **Management Commentary**:
   - "blah ceo whah says company had slow start to 2020"
   - "blah off to slow start in 2020; investors skeptical on lbo"

5. **Analyst Actions**:
   - "blah pt cut to $56 from $59 at mizuho securities usa" (price target reduction)

6. **Operational Concerns**:
   - "blah directed consultants to remove damaging findings: nyt"
   - "blah had consultants remove findings about mistakes: nyt"

### COVID-19 Impact Factors:
While the company is implementing COVID-19 responses (testing sites, waiving delivery fees, adjusted store hours), these are likely to increase operational costs in the short term:
   - "blah waiving delivery fees for all eligible prescriptions"
   - "blah: temporary space at some stores for covid-19 testing"
   - "blah announces temporary changes to store operating hours"

### Forward-Looking Indicators:
   - "Preview blah 2Q: covid-19 impact to take center stage" suggests analysts are concerned about COVID-19's impact on upcoming results
   - "blah options imply elevated post-earnings volatility" indicates market uncertainty about future performance

## Conclusion
Given the recent earnings miss, underperforming retail metrics, management's acknowledgment of a slow start to 2020, and the additional costs associated with COVID-19 response measures, Blah is likely to see a decrease in earnings in the next quarter. The market sentiment reflected in stock price declines, analyst price target cuts, and elevated implied volatility further supports this negative outlook.

In [33]:
system_prompt = """You are a senior financial analyst and review your teams work. You are looking at a financial summary and news for 'blah'. Using the summaries only, critique the report and construct an alternative narrative. If the narrative is in agreement with the two reports, make clear your belief in the direction of earning. If in disagreement, state why you disagree. Think through your response. {financial_summary} \n {news_summary}"""
prompt_template = PromptTemplate.from_template(system_prompt)
# Create the prompt to feed into the model
prompt_in = prompt_template.format(financial_summary=output_financial_statements.content, news_summary=output_news.content)

In [34]:
prompt_in

'You are a senior financial analyst and review your teams work. You are looking at a financial summary and news for \'blah\'. Using the summaries only, critique the report and construct an alternative narrative. If the narrative is in agreement with the two reports, make clear your belief in the direction of earning. If in disagreement, state why you disagree. Think through your response. # Financial Analysis Report\n\n## 1. Analysis of Current Profitability, Liquidity, Solvency, and Efficiency Ratios\n\n### Profitability Ratios\n- **Gross Profit Margin**: 21.24% (t) vs 21.61% (t-1)\n- **Operating Profit Margin**: 3.06% (t) vs 3.25% (t-1)\n- **Net Profit Margin**: 2.52% (t) vs 2.69% (t-1)\n- **Return on Assets (ROA)**: 3.88% (t) vs 4.08% (t-1)\n- **Return on Equity (ROE)**: 14.35% (t) vs 15.23% (t-1)\n\n### Liquidity Ratios\n- **Current Ratio**: 0.66 (t) vs 0.66 (t-1)\n- **Quick Ratio**: 0.32 (t) vs 0.31 (t-1)\n- **Cash Ratio**: 0.03 (t) vs 0.03 (t-1)\n\n### Solvency Ratios\n- **Debt-t

In [35]:
final_output = llm.invoke(prompt_in)

In [36]:
display(Markdown(final_output.content))

# Financial Analysis Review: Alternative Narrative for Blah

## Executive Summary

After reviewing both the financial analysis report and the news summary for Blah, I find significant concerns with the original analysis. While both reports correctly identify a negative earnings trajectory, I believe the original financial analysis underestimates the severity of Blah's situation and fails to properly contextualize the immediate challenges facing the company.

## Critical Assessment of Original Analysis

The original financial analysis presents a methodical review of Blah's financial metrics showing deteriorating performance across multiple dimensions. However, it has several shortcomings:

1. **Insufficient weight on recent performance**: The analysis focuses too heavily on 5-year trends while underemphasizing the acceleration of negative indicators in the most recent quarter.

2. **Inadequate consideration of COVID-19 impact**: While mentioned in the news summary, the original analysis fails to incorporate how the pandemic will likely exacerbate existing negative trends.

3. **Overly conservative earnings decline projection**: The projected 5-8% earnings decrease appears optimistic given the magnitude of recent misses and operational challenges.

4. **Insufficient attention to operational issues**: The news summary reveals concerning operational problems (consultants directed to remove damaging findings) that suggest deeper organizational issues beyond financial metrics.

## Alternative Narrative

Blah is experiencing not just a gradual decline but rather an accelerating deterioration in its business fundamentals that is likely to result in more significant earnings challenges than projected in the original analysis.

### Key Points Supporting This Alternative View:

1. **Accelerating Revenue and Earnings Pressure**
   - The recent quarterly miss (EPS of $1.37 vs. $1.41 expected) indicates the decline is worsening
   - US retail pharmacy comparable sales at just +1.6% versus +5.3% expected represents a dramatic underperformance in a core business segment
   - The 6% pre-market stock drop suggests investors see these misses as part of a worsening trend, not an isolated event

2. **Operational and Governance Red Flags**
   - The New York Times reporting about consultants being directed to remove damaging findings suggests potential governance issues that could have material impacts beyond current financials
   - This type of behavior often precedes more significant financial revelations and regulatory scrutiny

3. **COVID-19 as a Multiplier of Existing Problems**
   - The company's COVID-19 responses (waiving delivery fees, testing sites, adjusted hours) will create additional cost pressures
   - These initiatives come at a time when the company is already struggling with margin compression (gross margin declined from 23.10% to 21.24% over five years)
   - The company's already concerning liquidity position (current ratio of 0.66) may be further strained by pandemic-related disruptions

4. **Concerning Debt Position in a Rising Rate Environment**
   - The debt-to-equity ratio has increased from 1.66 to 2.70 over five years
   - The interest coverage ratio has declined from 9.79 to 6.15
   - In a potentially rising interest rate environment, this debt burden could become increasingly problematic

## Earnings Projection

I strongly disagree with the original analysis's projection of a 5-8% earnings decrease. Given the combination of:
- Accelerating negative trends in core business metrics
- Additional costs from COVID-19 response measures
- Potential governance issues suggested by the NYT reporting
- Significant analyst price target reductions
- Elevated implied volatility in options markets

I project that earnings will decrease by 12-15% in the next quarter, with potential for even greater deterioration if operational issues persist or if broader economic conditions worsen.

## Confidence Level: 90%

My higher confidence stems from the alignment between deteriorating financial fundamentals and the market's increasingly negative assessment, as evidenced by analyst downgrades, stock price declines, and elevated implied volatility.

## Strategic Recommendations

1. Closely monitor cash flow and liquidity metrics given the concerning current ratio
2. Investigate the operational issues highlighted in the NYT reporting for potential material impacts
3. Assess the sustainability of the dividend given declining profitability
4. Evaluate the effectiveness of the company's COVID-19 response strategies against their financial impact
5. Consider the implications of the company's high debt levels should interest rates rise

The original analysis correctly identified a negative trajectory but failed to fully appreciate the convergence of financial weakness, operational challenges, and external pressures that are likely to result in more significant earnings deterioration than initially projected.

## Langgraph Debate Step

In [50]:
from langgraph.graph import START, END, StateGraph
from langgraph.graph.message import add_messages

from typing import Dict, TypedDict, Optional

In [51]:
def llm_debate_invoke (prompt):
    return llm_debate.invoke(prompt).content

In [52]:
# number of agents
agents = ['BUY', 'SELL', 'HOLD']

In [154]:
class GraphState(TypedDict):
    classification: Optional[str] = None
    last_agent: Optional[str] = None
    history: Optional[str] = None
    summary_buy: Optional[str] = None
    summary_sell: Optional[str] = None
    summary_hold: Optional[str] = None
    current_response: Optional[str] = None
    count: Optional[int] = None
    results: Optional[str] = None
    consensus: Optional[dict[str,str]]

analyst_workflow = StateGraph(GraphState)

In [155]:
debate_agent_initial_system_prompt = """You are part of the investment committee at an asset management firm. Your senior analyst in the {sector} sector has presented the findings of reports with your fellow committee members until you come to an agreement on whether to recommend BUY, SELL or HOLD based on the stock price below and the reports from the analysts. If you are all in agreement then stop. You are advocating to {decision} the stock. Give a short 200 word summary of your decision. Only use the following context. Think through the response. Context: {senior_analyst_report} {financial_statement} {stock_prices}"""
agentic_initial_template = PromptTemplate.from_template(debate_agent_initial_system_prompt)

debate_agent_system_prompt = """You are part of the investment committee at an asset management firm. Your senior analyst in the {sector} sector has presented the findings of reports with your fellow committee members until you come to an agreement on whether to recommend {decision}. Do not repeat any previous arguments. Base your response only on the report and current conversation. If you do not recommend {decision}, state which direction you advocate. Provide the argument in less than 200 words. Think through your analysis. For review: {senior_analyst_report} \nPrior conversation: {conversation} """
agentic_debate_template = PromptTemplate.from_template(debate_agent_system_prompt)

debate_direction_system_prompt = """Based on the response, return the sentiment of the response as BUY, SELL or HOLD. Only return a single word: BUY, SELL or HOLD. Conversation: {conversation}"""
debate_direction_template = PromptTemplate.from_template(debate_direction_system_prompt)

result_system_prompt = """You are part of the investment committee at an asset management firm. You hold the deciding vote on whether to BUY, SELL or HOLD a stock. You must always proceed with the majority decision of the analysts. If there is no majority decision, decide on a winner. There can only be one decision. Only output the decision. The conversation is: {conversation}"""
results_template = PromptTemplate.from_template(result_system_prompt)

In [156]:
def debate_format(state, state_update, decision):
    """
    Function to run the debate between three agents. All use the same template/ format
    state_update: str - pass name of object to store the agent summary
    decision: str - identity of the agent
    """
    summary = state.get('history', '').strip()
    summary_x = state.get(state_update, '').strip()
    current_response = state.get('current_response', '').strip()
    if summary_x=='Nothing':
        # this is the initial argument
        prompt_in = agentic_initial_template.format(decision=decision, 
                                                    senior_analyst_report=final_output.content,
                                                    financial_statement=output_financial_statements.content,
                                                    stock_prices=sec_prices,
                                                    sector=sector)
        argument = decision + ":" + llm_debate_invoke(prompt_in)
        if summary == 'Nothing':
            return {'history': 'START\n' + argument, 
                    state_update: argument, 
                    'current_response': argument, 
                    'count':state.get('count')+1,
                    'last_agent': decision}
        else:
            return {'history': summary + '\n' + argument, 
                    state_update: argument, 
                    'current_response': argument, 
                    'count':state.get('count')+1,
                    'last_agent': decision}
    else:
        # this runs during the debate
        prompt_in = agentic_debate_template.format(decision=decision,
                                                      sector=sector,
                                                      senior_analyst_report=final_output.content,
                                                      conversation=summary)
        argument = decision + ":" + llm_debate_invoke(prompt_in)

        return {'history': summary + '\n' + argument,
                'current_response': argument, 
                'count': state.get('count') + 1,
                'last_agent': decision}

def next_node(state):
    last_agent = state.get('last_agent').strip()
    response_from_agent = state.get('current_response')
    current_consensus = state.get('consensus')
    prompt_in = debate_direction_template.format(conversation=response_from_agent)
    last_node = llm_debate_invoke(prompt_in)
    print(last_agent + ":" + last_node)
    current_consensus[last_agent] = last_node
    return {'consensus': current_consensus}

def handle_buy(state):
    return debate_format(state, 'summary_buy', 'BUY')

def handle_sell(state):
    return debate_format(state, 'summary_sell', 'SELL')

def handle_hold(state):
    return debate_format(state, 'summary_hold', 'HOLD')

def result(state):
    summary = state.get('history').strip()
    prompt_in = results_template.format(conversation=summary)
    return {"results":llm_debate_invoke(prompt_in)}

In [157]:
analyst_workflow.add_node('next_node', next_node)
analyst_workflow.add_node('handle_buy', handle_buy)
analyst_workflow.add_node('handle_sell', handle_sell)
analyst_workflow.add_node('handle_hold', handle_hold)
analyst_workflow.add_node('result', result)

In [158]:
# Add the logic to decide the next steps in the debate
def decide_next_node(state):
    last_agent = state.get('last_agent')
    current_consensus = state.get('consensus')
    if state.get('summary_sell', '') == 'Nothing':
        return 'handle_sell'
    if state.get('summary_hold', '') == 'Nothing':
        return 'handle_hold'

    if len(set(current_consensus.values())) == 1:
        # all the agents are in consensus
        return 'result'
    else:
        # agent is agreeing with their classification.
        # debate should continue
        if last_agent == 'BUY':
            return 'handle_sell'
        if last_agent == 'SELL':
            return 'handle_hold'
        if last_agent == 'HOLD':
            return 'handle_buy'

def check_conv_length(state):
    return "result" if state.get("count")==12 else "next_node"

analyst_workflow.add_conditional_edges(
    "next_node",
    decide_next_node,
    {
        "handle_buy": "handle_buy",
        "handle_sell": "handle_sell",
        "handle_hold": "handle_hold",
        "result": "result"
    }
)

analyst_workflow.add_conditional_edges(
    "handle_buy",
    check_conv_length,
    {
        "result": "result",
        "next_node": "next_node"
    }
)

analyst_workflow.add_conditional_edges(
    "handle_sell",
    check_conv_length,
    {
        "result": "result",
        "next_node": "next_node"
    }
)

analyst_workflow.add_conditional_edges(
    "handle_hold",
    check_conv_length,
    {
        "result": "result",
        "next_node": "next_node"
    }
)

In [159]:
analyst_workflow.set_entry_point("handle_buy")
analyst_workflow.add_edge('result', END)

In [160]:
app = analyst_workflow.compile()

In [161]:
app.get_graph()

Graph(nodes={'__start__': Node(id='__start__', name='__start__', data=<class 'langchain_core.utils.pydantic.LangGraphInput'>, metadata=None), 'next_node': Node(id='next_node', name='next_node', data=next_node(tags=None, recurse=True, explode_args=False, func_accepts_config=False, func_accepts={}), metadata=None), 'handle_buy': Node(id='handle_buy', name='handle_buy', data=handle_buy(tags=None, recurse=True, explode_args=False, func_accepts_config=False, func_accepts={}), metadata=None), 'handle_sell': Node(id='handle_sell', name='handle_sell', data=handle_sell(tags=None, recurse=True, explode_args=False, func_accepts_config=False, func_accepts={}), metadata=None), 'handle_hold': Node(id='handle_hold', name='handle_hold', data=handle_hold(tags=None, recurse=True, explode_args=False, func_accepts_config=False, func_accepts={}), metadata=None), 'result': Node(id='result', name='result', data=result(tags=None, recurse=True, explode_args=False, func_accepts_config=False, func_accepts={}), m

In [162]:
%%time
initial_state = {
    'count':0,
    'history':'Nothing',
    'current_response':'',
    'summary_buy': 'Nothing',
    'summary_sell': 'Nothing',
    'summary_hold': 'Nothing',
    'consensus': {'BUY':'BUY', 'SELL':'SELL', 'HOLD':'HOLD'}
}

conversation = app.invoke(initial_state)

BUY:BUY
SELL:SELL
HOLD:HOLD
BUY:SELL
SELL:SELL
HOLD:HOLD
BUY:BUY
SELL:SELL
HOLD:HOLD
BUY:SELL
SELL:SELL
CPU times: user 169 ms, sys: 0 ns, total: 169 ms
Wall time: 1min 39s


In [163]:
print(conversation['results'])

SELL


In [164]:
print(conversation['history'])

START
BUY:Based on the comprehensive analysis and context provided, I will advocate for a BUY recommendation to the investment committee:

Investment Recommendation: BUY

Our analysis reveals a nuanced opportunity that the market may be undervaluing. While the financial metrics show concerning trends, we believe the current stock price presents an attractive entry point for strategic investors.

Key rationale for our BUY recommendation:

1. Valuation Opportunity: The stock has experienced a significant 27.17% price decline, creating an attractive entry point. The current price of $40.32 appears to be oversold, considering the company's underlying fundamentals.

2. Operational Resilience: Despite challenging conditions, the company has maintained revenue stability and demonstrated improved inventory management. The asset turnover ratio remains consistent, indicating operational efficiency.

3. Potential Turnaround Indicators: 
- Improved inventory turnover (9.43 to 11.32)
- Stable asset

In [167]:
print(conversation['consensus'])

{'BUY': 'SELL', 'SELL': 'SELL', 'HOLD': 'HOLD'}


In [166]:
conversation['results']

'SELL'